In [1]:
date='0222'
filenumber='9'  #16

In [2]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras import regularizers
from keras.utils import plot_model

Using TensorFlow backend.


In [3]:
from keras.layers import Dense, Input, Dropout, Activation, BatchNormalization, Concatenate, Conv1D, Flatten, MaxPooling1D
from keras.models import Model, load_model
from keras import optimizers, callbacks
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras import backend as K
from swa.keras import SWA
from keras import initializers


In [4]:
# 데이터 불러오기
train = pd.read_csv('/home/lab21/data/train.csv', index_col=0)
train_2 = pd.read_csv('/home/lab21/data/train_x_0.2_99.8.csv', index_col=0)

test = pd.read_csv('/home/lab21/data/test.csv', index_col=0)
sample_submission = pd.read_csv('/home/lab21/data/sample_submission.csv', index_col=0)

# Train 데이터의 타입을 Sample_submission에 대응하는 가변수 형태로 변환
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train['type_num'] = train['type'].apply(lambda x : to_number(x, column_number))

# 모델에 적용할 데이터 셋 준비 
#x = train.drop(columns=['type', 'type_num'], axis=1)
y = train['type_num']

x = train_2.drop(columns=['fiberID'], axis=1)

test_x = test.drop(columns=['fiberID'],axis=1)

x_name=x.columns
col_name=x_name

In [5]:
x=np.array(x)
y=np.array(y)
test_x=np.array(test_x)


In [6]:
x=x.astype('float32')
test_x=test_x.astype('float32')

In [7]:
# # Train_test_split
# In[7]:

from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit

x_train, x_val, y_train, y_val = train_test_split(x,y, stratify=y, \
                                                    train_size=0.6, shuffle=True ,random_state=0)


In [8]:

from sklearn.preprocessing import StandardScaler, RobustScaler

# scaler1=RobustScaler()
scaler2=StandardScaler()

scaler2.fit(x_train)
x_train=scaler2.transform(x_train)
x_val=scaler2.transform(x_val)

test_x =scaler2.transform(test_x)

from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_val = to_categorical(y_val)


In [9]:
import math
from keras.callbacks import Callback

class LearningRateScheduler(Callback):
    """LearningRateScheduler"""
    
    def __init__(self, T_max, eta_max, eta_min, decay=10000,verbose=0):
        super(LearningRateScheduler, self).__init__()
        self.T_max = T_max
        self.eta_max = eta_max
        self.eta_min = eta_min
        self.decay = decay
        self.verbose = verbose
        
    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, 'lr'):
            raise ValueError('Optimizer must have a "lr" attribute.')

        x1=1.
        x2=1.-(2.*self.eta_min)/(self.eta_max+self.eta_min)
        x3=x1+x2
        
        lr=self.eta_max/(x3)*(x1-x2*(np.cos(math.pi * np.mod(epoch,self.T_max) / (self.T_max) +np.pi))) * np.power(0.5,epoch/self.decay)

        K.set_value(self.model.optimizer.lr, lr)
        if self.verbose > 0:
            print('\nEpoch %05d: LearningRateSchedule setting learning '
                  'rate to %s.' % (epoch + 1, lr))

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        logs['lr'] = K.get_value(self.model.optimizer.lr)

In [10]:
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects
import tensorflow as tf

class Gelu(Activation):
    def __init__(self, activation, **kwargs):
        super(Gelu, self).__init__(activation, **kwargs)
        self.__name__='gelu'
        
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * np.power(x, 3))))

get_custom_objects().update({'gelu': Gelu(gelu)})

In [11]:
lr = 2.e-5
# lr_d = 0
patience =  40
drop = 0.05
nepoch = 240*4

In [12]:
T_max=60
eta_max=2.0e-5
eta_min=2.0e-5
dec=0.000000001
decay=nepoch/dec

In [13]:
early_stop = EarlyStopping(monitor="val_loss", mode="min", patience=patience)
learning_scheduler = LearningRateScheduler(T_max=T_max, eta_max=eta_max, eta_min=eta_min, decay=decay, verbose=1)

import datetime
start=datetime.datetime.now()
print(start)

2020-02-22 10:00:36.253268


In [14]:
from keras.utils import plot_model

In [15]:
x1_train=x_train.reshape(-1,20,1)
x1_val= x_val.reshape(-1,20,1)


In [16]:

inps1 = Input(shape = (20,1))
inps2 = Input(shape = (20,))

node = 128 # 128
#############################################################

la1 = Conv1D(128, 5, activation='relu',padding='same', strides=5)(inps1)
#la1 = Conv1D(64, 5, activation='relu',padding='same')(la1)
x1 = Flatten()(la1)

for i in range(2):
    xp = Dense(node)(x1)

    x1 = Dense(node*2)(x1)
    x1 = Activation('relu')(x1)
    x1 = Dropout(0.20)(x1)

    xq= Dense(node)(x1)

    x1 = Concatenate()([xp,xq])
    x1 = Activation('relu')(x1)


##############################################################

la2 = Dense(node)(inps2)
x2  = Activation('relu')(la2)

for i in range(2):
    xp = Dense(node)(x2)

    x2 = Dense(node*2)(x2)
    x2 = Activation('relu')(x2)
    x2 = Dropout(0.20)(x2)

    xq= Dense(node)(x2)

    x2 = Concatenate()([xp,xq])
    x2 = Activation('relu')(x2)


x4 = Concatenate()([x1,x2])

x4 = Dense(node)(x4)
x4 = Activation('relu')(x4)
x4 = Dropout(drop)(x4)

x4 = Dense(node)(x4)
x4 = Activation('relu')(x4)
x4 = Dropout(drop)(x4)

outs=Dense(19,activation='softmax',name='output')(x4)

models = Model(inputs=[inps1,inps2], outputs=outs)
models.fit
models.compile(loss='categorical_crossentropy', optimizer=Adam(lr=lr),metrics=['accuracy'])

models.summary()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 20, 1)        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 20)           0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 4, 128)       768         input_1[0][0]                    
__________________________________________________________________________________________________
dense_7 (Dense)      

In [ ]:

hist = models.fit([x1_train,x_train], y_train, batch_size= 256, epochs=nepoch, verbose=2, validation_data=[[x1_val,x_val], y_val], callbacks=[early_stop,learning_scheduler]) #,swa

loss, acc = models.evaluate([x1_test,x_test],y_test,batch_size= 256)
print('Loss:',loss,'Accuracy:',acc)


Instructions for updating:
Use tf.cast instead.
Train on 119994 samples, validate on 79997 samples
Epoch 1/960

Epoch 00001: LearningRateSchedule setting learning rate to 2e-05.
 - 12s - loss: 1.8006 - acc: 0.5099 - val_loss: 1.0825 - val_acc: 0.6671
Epoch 2/960

Epoch 00002: LearningRateSchedule setting learning rate to 1.999999999998556e-05.
 - 10s - loss: 1.0072 - acc: 0.6849 - val_loss: 0.8705 - val_acc: 0.7387
Epoch 3/960

Epoch 00003: LearningRateSchedule setting learning rate to 1.999999999997112e-05.
 - 10s - loss: 0.8804 - acc: 0.7267 - val_loss: 0.7703 - val_acc: 0.7760
Epoch 4/960

Epoch 00004: LearningRateSchedule setting learning rate to 1.999999999995668e-05.
 - 10s - loss: 0.7836 - acc: 0.7640 - val_loss: 0.6869 - val_acc: 0.7976
Epoch 5/960

Epoch 00005: LearningRateSchedule setting learning rate to 1.9999999999942237e-05.
 - 11s - loss: 0.7110 - acc: 0.7856 - val_loss: 0.6314 - val_acc: 0.8124
Epoch 6/960

Epoch 00006: LearningRateSchedule setting learning rate to 1.99

In [ ]:
end=datetime.datetime.now()
print(end)
print("걸린 시간:", end-start)

In [ ]:
# #y_pred = models.predict_proba(test_x)

# test1_x=test_x.reshape(-1,20,1)
# y_pred = models.predict([test1_x,test_x])

# # 제출 파일 생성
# submission = pd.DataFrame(data=y_pred, columns=sample_submission.columns, index=sample_submission.index)
# submission.to_csv('/home/lab21/data/submission_no_fold_'+date+filenumber+'.csv', index=True)

In [ ]:
import matplotlib.pyplot as plt

print(hist.history.keys())

fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='center')

acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='center right')

plt.savefig('/home/lab21/data/submission_no_fold_'+date+filenumber+'_1.png')
plt.show()

fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
loss_ax.legend(loc='lower right')
loss_ax.set_xlim(-1,1000)
loss_ax.set_ylim(0.3,0.4)
plt.savefig('/home/lab21/data/submission_no_fold_tr_'+date+filenumber+'_2.png')
plt.show()

fig, loss_ax = plt.subplots()
acc_ax = loss_ax.twinx()

acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')
acc_ax.set_ylabel('accuracy')
acc_ax.legend(loc='center right')
acc_ax.set_xlim(100,1000)
acc_ax.set_ylim(0.86,0.88)
plt.savefig('/home/lab21/data/submission_no_fold_tr_'+date+filenumber+'_3.png')
plt.show()

plt.plot(hist.history['lr'])
plt.savefig('/home/lab21/data/submission_no_fold_tr_'+date+filenumber+'_lr.png')
plt.show()